# Lasso Logistic Regression

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
data=pd.read_csv("train.csv",nrows=50000)
data.shape

(50000, 133)

In [3]:
data.head()

,ID,target,v1,v2,v3,v4,v5,v6,v7,v8,...,v122,v123,v124,v125,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,C,3.921026,7.915266,2.599278,3.176895,0.012941,...,8.000000,1.989780,0.035754,AU,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,C,NaN,9.191265,NaN,NaN,2.301630,...,NaN,NaN,0.598896,AF,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,C,4.410969,5.326159,3.979592,3.928571,0.019645,...,9.333333,2.477596,0.013452,AE,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,C,4.225930,11.627438,2.097700,1.987549,0.171947,...,7.018256,1.812795,0.002267,CJ,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,C,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,Z,NaN,NaN,NaN,0,NaN,NaN


In [4]:
numerics = ['int16', 'int32', 'int64', 'float16', 'float32', 'float64']
numerical_vars = list(data.select_dtypes(include=numerics).columns)
data = data[numerical_vars]
data.shape

(50000, 114)

In [5]:
data.head()

,ID,target,v1,v2,v4,v5,v6,v7,v8,v9,...,v121,v122,v123,v124,v126,v127,v128,v129,v130,v131
0,3,1,1.335739,8.727474,3.921026,7.915266,2.599278,3.176895,0.012941,9.999999,...,0.803572,8.000000,1.989780,0.035754,1.804126,3.113719,2.024285,0,0.636365,2.857144
1,4,1,NaN,NaN,NaN,9.191265,NaN,NaN,2.301630,NaN,...,NaN,NaN,NaN,0.598896,NaN,NaN,1.957825,0,NaN,NaN
2,5,1,0.943877,5.310079,4.410969,5.326159,3.979592,3.928571,0.019645,12.666667,...,2.238806,9.333333,2.477596,0.013452,1.773709,3.922193,1.120468,2,0.883118,1.176472
3,6,1,0.797415,8.304757,4.225930,11.627438,2.097700,1.987549,0.171947,8.965516,...,1.956521,7.018256,1.812795,0.002267,1.415230,2.954381,1.990847,1,1.677108,1.034483
4,8,1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,NaN,NaN


In [6]:
from sklearn.model_selection import train_test_split

In [7]:
X_train, X_test, y_train, y_test = train_test_split(data.drop(labels=['target', 'ID'], axis=1),
                                                    data['target'],
                                                    test_size=0.3,
                                                    random_state=0)

print(X_train.shape,y_train.shape)
print(X_test.shape,y_test.shape)

(35000, 112) (35000,)
(15000, 112) (15000,)


#### linear models benefit from feature scaling


In [8]:
from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
scaler.fit(X_train.fillna(0))

StandardScaler(copy=True, with_mean=True, with_std=True)

In [9]:
X_train.head()

,v1,v2,v4,v5,v6,v7,v8,v9,v10,v11,...,v121,v122,v123,v124,v126,v127,v128,v129,v130,v131
17967,1.947369,6.969209,2.823776,8.394091,1.563910,2.887218,0.344201,11.694915,1.575492,14.982332,...,3.680982,8.305084,3.444048,1.261967e-01,1.486649,6.118421,1.380593,2,1.833333,1.590909
32391,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1.816192,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2,NaN,NaN
9341,1.827749,7.587996,3.344134,NaN,3.324743,1.799828,NaN,9.010600,1.575493,16.170569,...,2.553801,6.749116,NaN,NaN,1.399995,0.708762,NaN,0,2.529832,1.735848
7929,0.522894,6.551627,6.141825,10.209257,2.634256,2.719051,0.862119,8.712872,3.916849,15.454545,...,1.391863,6.633664,1.844494,2.505026e-01,2.204730,2.734595,3.090974,2,0.923078,0.900900
46544,2.086927,7.053496,3.801169,5.884332,2.070339,2.906437,0.132119,9.322494,1.312909,14.919093,...,3.481894,5.853658,3.946788,6.090014e-07,2.076193,4.615960,1.242660,0,1.890411,1.642512


Let us use LogisticRegression and L1 penalty

In [12]:
from sklearn.linear_model import Lasso,LogisticRegression
from sklearn.feature_selection import SelectFromModel

In [13]:
sel_ = SelectFromModel(LogisticRegression(C=1, penalty='l1'))
sel_.fit(scaler.transform(X_train.fillna(0)), y_train)

SelectFromModel(estimator=LogisticRegression(C=1, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
        prefit=False, threshold=None)

In [22]:
selected_feat = X_train.columns[(sel_.get_support())]
removed_feats = X_train.columns[(sel_.estimator_.coef_ == 0).ravel()]

In [24]:
len(removed_feats)

33

33 feature coffecients were zero by L1 penalty. Based on C value the number of coefficients becoming zero will change